<a href="https://colab.research.google.com/github/KetkiGupta99/flight-delay-prediction/blob/main/flight_delay_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
divyansh22_flight_delay_prediction_path = kagglehub.dataset_download('divyansh22/flight-delay-prediction')

print('Data source import complete.')

100%|██████████| 22.9M/22.9M [00:00<00:00, 104MB/s] 

Extracting files...


Data source import complete.


In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [3]:
#Installing PySpark
!pip install pyspark --quiet

In [4]:
#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, when
from pyspark.ml.feature import VectorAssembler
# Import the Decision Tree Classifier class
from pyspark.ml.classification  import DecisionTreeClassifier
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
#Building Spark Session
spark = (SparkSession.builder
                  .appName('flight')
                  .config("spark.executor.memory", "1G")
                  .config("spark.executor.cores","4")
                  .getOrCreate())

spark.sparkContext.setLogLevel('WARN')
spark.version

'3.5.1'

In [7]:
# Read data from CSV file
flights = spark.read.csv(os.path.join(divyansh22_flight_delay_prediction_path, 'Jan_2019_ontime.csv'),
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
print(flights.dtypes)
flights.count()

The data contain 583985 records.
+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+----+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|OP_CARRIER_AIRLINE_ID|OP_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST|DEP_TIME|DEP_DEL15|DEP_TIME_BLK|ARR_TIME|ARR_DEL15|CANCELLED|DIVERTED|DISTANCE|_c21|
+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+----+
|           1|          2|               9E|                20363|        9E|  N8688C|             3280|            11953|              

583985

In [8]:
#Drop similar columns
flights=flights.drop('OP_UNIQUE_CARRIER','OP_CARRIER','TAIL_NUM','ORIGIN_AIRPORT_SEQ_ID','ORIGIN','DEST_AIRPORT_SEQ_ID','DEST','_c21','DEP_TIME_BLK')
flights.show(5)

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+---------+--------+--------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|DEP_DEL15|ARR_TIME|ARR_DEL15|CANCELLED|DIVERTED|DISTANCE|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+---------+--------+--------+
|           1|          2|                20363|             3280|            11953|          10397|     601|      0.0|     722|      0.0|      0.0|     0.0|   300.0|
|           1|          2|                20363|             3281|            13487|          11193|    1359|      0.0|    1633|      0.0|      0.0|     0.0|   596.0|
|           1|          2|                20363|             3282|            11433|          11193|    1215|      0.0|    1329|      0.0|      0.0|     0.0|   229.0

In [9]:
# Create a list of column names
column_names = flights.columns
# Calculate the count of missing values for each column
missing_value_counts = [ flights.where(col(column_name).isNull()).count() for column_name in column_names]

# Create a DataFrame to display the results
result_flights = spark.createDataFrame(list(zip(column_names, missing_value_counts)), ["Column", "Missing_Values_Count"])

# Show the result
result_flights.show()

+--------------------+--------------------+
|              Column|Missing_Values_Count|
+--------------------+--------------------+
|        DAY_OF_MONTH|                   0|
|         DAY_OF_WEEK|                   0|
|OP_CARRIER_AIRLIN...|                   0|
|   OP_CARRIER_FL_NUM|                   0|
|   ORIGIN_AIRPORT_ID|                   0|
|     DEST_AIRPORT_ID|                   0|
|            DEP_TIME|               16352|
|           DEP_DEL15|               16355|
|            ARR_TIME|               17061|
|           ARR_DEL15|               18022|
|           CANCELLED|                   0|
|            DIVERTED|                   0|
|            DISTANCE|                   0|
+--------------------+--------------------+



In [10]:
#Drop records with missing values
flights=flights.dropna()
flights.count()

565963

In [11]:
flights.show(5)

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+---------+--------+--------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|DEP_DEL15|ARR_TIME|ARR_DEL15|CANCELLED|DIVERTED|DISTANCE|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+---------+--------+--------+
|           1|          2|                20363|             3280|            11953|          10397|     601|      0.0|     722|      0.0|      0.0|     0.0|   300.0|
|           1|          2|                20363|             3281|            13487|          11193|    1359|      0.0|    1633|      0.0|      0.0|     0.0|   596.0|
|           1|          2|                20363|             3282|            11433|          11193|    1215|      0.0|    1329|      0.0|      0.0|     0.0|   229.0

In [12]:
# Calculate the frequency of data in the "CANCELLED" column
frequency_canceled = flights.groupBy("CANCELLED").count()

# Show the result
frequency_canceled.show()

+---------+------+
|CANCELLED| count|
+---------+------+
|      0.0|565963|
+---------+------+



In [13]:
# Calculate the frequency of data in the "DEP_DEL15" column
frequency_DEP_DEL15 = flights.groupBy("DEP_DEL15").count()
# Show the result
frequency_DEP_DEL15.show()

+---------+------+
|DEP_DEL15| count|
+---------+------+
|      0.0|467658|
|      1.0| 98305|
+---------+------+



In [14]:
# Calculate the frequency of data in the "ARR_DEL15" column
frequency_ARR_DEL15 = flights.groupBy("ARR_DEL15").count()
# Show the result
frequency_ARR_DEL15.show()

+---------+------+
|ARR_DEL15| count|
+---------+------+
|      0.0|460741|
|      1.0|105222|
+---------+------+



In [15]:
# Calculate the frequency of data in the "DIVERTED" column
frequency_DIVERTED = flights.groupBy("DIVERTED").count()
# Show the result
frequency_DIVERTED.show()

+--------+------+
|DIVERTED| count|
+--------+------+
|     0.0|565963|
+--------+------+



In [16]:
# Calculate the frequency of data in the "OP_CARRIER_AIRLINE_ID" column
frequency_OP_CARRIER_AIRLINE_ID = flights.groupBy("OP_CARRIER_AIRLINE_ID").count()
# Show the result
frequency_OP_CARRIER_AIRLINE_ID.show()

+---------------------+------+
|OP_CARRIER_AIRLINE_ID| count|
+---------------------+------+
|                20398| 23710|
|                19690|  6785|
|                20409| 23395|
|                20452| 24569|
|                20378| 17829|
|                19790| 73403|
|                20304| 61770|
|                20436|  9490|
|                19393|107235|
|                19805| 75354|
|                20368|  6694|
|                19930| 20204|
|                20366| 12200|
|                20363| 19821|
|                19977| 46126|
|                20397| 22369|
|                20416| 15009|
+---------------------+------+



In [17]:
#Drop useless columns
flights=flights.drop('CANCELLED','DIVERTED')
flights.show(5)

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+--------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|DEP_DEL15|ARR_TIME|ARR_DEL15|DISTANCE|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+--------+
|           1|          2|                20363|             3280|            11953|          10397|     601|      0.0|     722|      0.0|   300.0|
|           1|          2|                20363|             3281|            13487|          11193|    1359|      0.0|    1633|      0.0|   596.0|
|           1|          2|                20363|             3282|            11433|          11193|    1215|      0.0|    1329|      0.0|   229.0|
|           1|          2|                20363|             3283|            15249|          10397|    1521|   

In [18]:
# Make one column for delay either arrival or departure
flights = flights.withColumn("Delay", when((col("DEP_DEL15") >= 1.0) | (col("ARR_DEL15") >= 1.0), 1).otherwise(0))
flights.show(5)
# Check column data types
print(flights.dtypes)
flights.count()

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+--------+-----+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|DEP_DEL15|ARR_TIME|ARR_DEL15|DISTANCE|Delay|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+---------+--------+---------+--------+-----+
|           1|          2|                20363|             3280|            11953|          10397|     601|      0.0|     722|      0.0|   300.0|    0|
|           1|          2|                20363|             3281|            13487|          11193|    1359|      0.0|    1633|      0.0|   596.0|    0|
|           1|          2|                20363|             3282|            11433|          11193|    1215|      0.0|    1329|      0.0|   229.0|    0|
|           1|          2|                20363|             3283|          

565963

In [19]:
# Calculate the frequency of data in the "Delay" column
frequency_Delay = flights.groupBy("Delay").count()
# Show the result
frequency_Delay.show()

+-----+------+
|Delay| count|
+-----+------+
|    1|125239|
|    0|440724|
+-----+------+



In [20]:
#Drop old columns
flights=flights.drop('DEP_DEL15','ARR_DEL15')
flights.show(5)

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+--------+--------+-----+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|ARR_TIME|DISTANCE|Delay|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+--------+--------+-----+
|           1|          2|                20363|             3280|            11953|          10397|     601|     722|   300.0|    0|
|           1|          2|                20363|             3281|            13487|          11193|    1359|    1633|   596.0|    0|
|           1|          2|                20363|             3282|            11433|          11193|    1215|    1329|   229.0|    0|
|           1|          2|                20363|             3283|            15249|          10397|    1521|    1625|   223.0|    0|
|           1|          2|                20363|             3

In [21]:
#Assembling columns
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'DEP_TIME', 'ARR_TIME','DISTANCE'
], outputCol='features')

# Consolidate predictor columns
flights= assembler.transform(flights)
# Check the resulting column
flights.select('features', 'Delay').show(5, truncate=False)

+------------------------------------------------------------+-----+
|features                                                    |Delay|
+------------------------------------------------------------+-----+
|[1.0,2.0,20363.0,3280.0,11953.0,10397.0,601.0,722.0,300.0]  |0    |
|[1.0,2.0,20363.0,3281.0,13487.0,11193.0,1359.0,1633.0,596.0]|0    |
|[1.0,2.0,20363.0,3282.0,11433.0,11193.0,1215.0,1329.0,229.0]|0    |
|[1.0,2.0,20363.0,3283.0,15249.0,10397.0,1521.0,1625.0,223.0]|0    |
|[1.0,2.0,20363.0,3284.0,10397.0,11778.0,1847.0,1940.0,579.0]|0    |
+------------------------------------------------------------+-----+
only showing top 5 rows



In [22]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=43)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print(training_ratio)

0.8001901184353041


In [23]:
flights_train.show(5)

+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+--------+--------+-----+--------------------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|DEP_TIME|ARR_TIME|DISTANCE|Delay|            features|
+------------+-----------+---------------------+-----------------+-----------------+---------------+--------+--------+--------+-----+--------------------+
|           1|          2|                19393|                4|            10397|          14122|    1227|    1352|   526.0|    0|[1.0,2.0,19393.0,...|
|           1|          2|                19393|                4|            11259|          10397|     845|    1133|   721.0|    0|[1.0,2.0,19393.0,...|
|           1|          2|                19393|                4|            12191|          11259|     700|     751|   239.0|    0|[1.0,2.0,19393.0,...|
|           1|          2|                19393|                4|    

In [24]:
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier(labelCol="Delay",featuresCol='features')
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('Delay', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|Delay|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |0.0       |[0.897777881955839,0.10222211804416108] |
|0    |0.0       |[0.897777881955839,0.10222211804416108] |
|0    |0.0       |[0.897777881955839,0.10222211804416108] |
|0    |0.0       |[0.8068686285660932,0.19313137143390685]|
|0    |0.0       |[0.8068686285660932,0.19313137143390685]|
+-----+----------+----------------------------------------+
only showing top 5 rows



In [25]:
# Create a confusion matrix
prediction.groupBy('Delay', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND Delay = prediction').count()
TP = prediction.filter('prediction=1 AND Delay = prediction').count()
FN = prediction.filter('prediction = 0 AND Delay = 1').count()
FP = prediction.filter('prediction = 1 AND Delay= 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|Delay|prediction|count|
+-----+----------+-----+
|    1|       0.0|24126|
|    0|       0.0|87413|
|    1|       1.0| 1129|
|    0|       1.0|  417|
+-----+----------+-----+

0.7829685634699562


In [26]:
# Create a classifier object and train on training data
lr = LogisticRegression(featuresCol="features", labelCol="Delay")
lr_model = lr.fit(flights_train)
# Create predictions for the testing data and show confusion matrix
predictions = lr_model.transform(flights_test)
predictions.groupBy('Delay', 'prediction').count().show()

+-----+----------+-----+
|Delay|prediction|count|
+-----+----------+-----+
|    1|       0.0|25147|
|    0|       0.0|87678|
|    1|       1.0|  108|
|    0|       1.0|  152|
+-----+----------+-----+



In [27]:
evaluator = BinaryClassificationEvaluator(labelCol="Delay", rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5012728824760077
